<a href="https://colab.research.google.com/github/zegabr/pln-chatbot/blob/main/notebooks/extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#http://alexminnaar.com/2019/08/22/ner-rnns-tensorflow.html
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import numpy as np
import os
import time
from pandas import read_csv

print(tf.__version__)

2.5.0


In [1]:
TRAIN_DATASET_PATH = 'https://raw.githubusercontent.com/zegabr/pln-chatbot/main/dataset/extraction/train_dataset.csv'
TEST_DATASET_PATH = 'https://raw.githubusercontent.com/zegabr/pln-chatbot/main/dataset/extraction/test_dataset.csv'
VALID_DATASET_PATH = 'https://raw.githubusercontent.com/zegabr/pln-chatbot/main/dataset/extraction/dev_dataset.csv'

In [8]:
train_dataset = read_csv(TRAIN_DATASET_PATH)[:-1]
test_dataset = read_csv(TEST_DATASET_PATH)[:-1]
valid_dataset = read_csv(VALID_DATASET_PATH)[:-1]
print(train_dataset)
print(test_dataset)
print(valid_dataset)

            Phrase Entity
0                I      O
1             need      O
2                a      O
3       restaurant      O
4      reservation      O
...            ...    ...
32532        thank      O
32533          you      O
32534         very      O
32535         much      O
32536            !      O

[32537 rows x 2 columns]
          Phrase Entity
0              I      O
1             'm      O
2        looking      O
3            for      O
4              a      O
...          ...    ...
6366           .      O
6367      Really      O
6368  appreciate      O
6369          it      O
6370           !      O

[6371 rows x 2 columns]
      Phrase Entity
0         Do      O
1        you      O
2       know      O
3         of      O
4        any      O
...      ...    ...
7085       .      O
7086  Thanks      O
7087      so      O
7088    much      O
7089       .      O

[7090 rows x 2 columns]


In [33]:
from pandas import notna

labels = set()

def to_examples(dataset):
  examples=[]
  example = [[],[]]
  for idx, row in dataset.iterrows():
    phrase, entity = row['Phrase'], row['Entity']
    if notna(phrase) and notna(entity):
      example[0].append(phrase)
      example[1].append(entity)
      labels.add(entity)
    else:
      examples.append(example)
      example=[[],[]]
  return examples

train_examples = to_examples(train_dataset)
test_examples = to_examples(test_dataset)
valid_examples = to_examples(valid_dataset)
print(train_examples[:10])


[[['I', 'need', 'a', 'restaurant', 'reservation', '.'], ['O', 'O', 'O', 'O', 'O', 'O']], [['It', 'has', 'to', 'be', 'in', 'San', 'Fran', '.'], ['O', 'O', 'O', 'O', 'O', 'B-city', 'I-city', 'O']], [['It', "'ll", 'be', 'afternoon', '12', '.'], ['O', 'O', 'O', 'B-time', 'I-time', 'O']], [['I', "'d", 'like', 'Palmer', "'s", '.'], ['O', 'O', 'O', 'B-restaurant_name', 'I-restaurant_name', 'O']], [['Perfect', '.', 'What', "'s", 'their', 'cuisine', 'type', 'and', 'number', '?'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']], [['Thanks', ',', 'a', 'lot', '.', 'That', "'ll", 'do', 'it', ',', 'for', 'now', '.'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']], [['On', 'March', '2nd', 'we', 'are', 'going', 'to', 'celebrate', 'my', 'wedding', 'anniversary', '.', 'So', 'please', 'check', 'for', 'a', 'reservation', 'at', 'a', 'restaurant', '.'], ['O', 'B-date', 'I-date', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']], [['Yes', 

In [34]:
    # create character vocab
    all_text = " ".join([" ".join(x[0]) for x in train_examples+valid_examples+test_examples])
    vocab = sorted(set(all_text))
    
    # create character/id and label/id mapping
    char2idx = {u:i+1 for i, u in enumerate(vocab)}
    idx2char = np.array(vocab)
    label2idx = {u:i+1 for i, u in enumerate(labels)}
    idx2label = np.array(labels)
    
    print(idx2label)
    print(char2idx)

{'I-cuisine', 'B-city', 'B-location', 'B-restaurant_name', 'I-restaurant_name', 'I-price_range', 'B-price_range', 'I-category', 'B-party_size', 'I-location', 'I-date', 'I-city', 'B-time', 'B-cuisine', 'B-date', 'B-number_of_seats', 'O', 'B-category', 'I-time'}
{' ': 1, '!': 2, '&': 3, "'": 4, ',': 5, '-': 6, '.': 7, '/': 8, '0': 9, '1': 10, '2': 11, '3': 12, '4': 13, '5': 14, '6': 15, '7': 16, '8': 17, '9': 18, ':': 19, ';': 20, '?': 21, 'A': 22, 'B': 23, 'C': 24, 'D': 25, 'E': 26, 'F': 27, 'G': 28, 'H': 29, 'I': 30, 'J': 31, 'K': 32, 'L': 33, 'M': 34, 'N': 35, 'O': 36, 'P': 37, 'Q': 38, 'R': 39, 'S': 40, 'T': 41, 'U': 42, 'V': 43, 'W': 44, 'Y': 45, 'Z': 46, '`': 47, 'a': 48, 'b': 49, 'c': 50, 'd': 51, 'e': 52, 'f': 53, 'g': 54, 'h': 55, 'i': 56, 'j': 57, 'k': 58, 'l': 59, 'm': 60, 'n': 61, 'o': 62, 'p': 63, 'q': 64, 'r': 65, 's': 66, 't': 67, 'u': 68, 'v': 69, 'w': 70, 'x': 71, 'y': 72, 'z': 73}


In [35]:
    def split_char_labels(eg):
      '''
      For a given input/output example, break tokens into characters while keeping 
      the same label.
      '''

      tokens = eg[0]
      labels=eg[1]

      input_chars = []
      output_char_labels = []

      for token,label in zip(tokens,labels):

        input_chars.extend([char for char in token])
        input_chars.extend(' ')
        output_char_labels.extend([label]*len(token))
        output_char_labels.extend('O')

      return [[char2idx[x] for x in input_chars[:-1]],np.array([label2idx[x] for x in output_char_labels[:-1]])]
   
    train_formatted = [split_char_labels(eg) for eg in train_examples]
    test_formatted = [split_char_labels(eg) for eg in test_examples]
    valid_formatted = [split_char_labels(eg) for eg in valid_examples]
    
    print(len(train_formatted))
    print(len(test_formatted))
    print(len(valid_formatted))

2975
532
626


In [36]:
    # training generator
    def gen_train_series():

        for eg in train_formatted:
          yield eg[0],eg[1]
    
    # validation generator
    def gen_valid_series():
    
       for eg in valid_formatted:
          yield eg[0],eg[1]
    
    # test generator
    def gen_test_series():

      for eg in test_formatted:
          yield eg[0],eg[1]
      
    # create Dataset objects for train, test and validation sets  
    series = tf.data.Dataset.from_generator(gen_train_series,output_types=(tf.int32, tf.int32),output_shapes = ((None, None)))
    series_valid = tf.data.Dataset.from_generator(gen_valid_series,output_types=(tf.int32, tf.int32),output_shapes = ((None, None)))
    series_test = tf.data.Dataset.from_generator(gen_test_series,output_types=(tf.int32, tf.int32),output_shapes = ((None, None)))

    BATCH_SIZE = 128
    BUFFER_SIZE=1000
    
    # create padded batch series objects for train, test and validation sets
    ds_series_batch = series.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes=([None], [None]), drop_remainder=True)
    ds_series_batch_valid = series_valid.padded_batch(BATCH_SIZE, padded_shapes=([None], [None]), drop_remainder=True)
    ds_series_batch_test = series_test.padded_batch(BATCH_SIZE, padded_shapes=([None], [None]), drop_remainder=True)
    
    # print example batches
    for input_example_batch, target_example_batch in ds_series_batch_valid.take(1):
      print(input_example_batch)
      print(target_example_batch)

tf.Tensor(
[[25 62  1 ...  0  0  0]
 [30  1  4 ...  0  0  0]
 [45 52 48 ...  0  0  0]
 ...
 [41 55 48 ...  0  0  0]
 [30  1  4 ...  0  0  0]
 [30 61  1 ...  0  0  0]], shape=(128, 132), dtype=int32)
tf.Tensor(
[[17 17 17 ...  0  0  0]
 [17 17 17 ...  0  0  0]
 [17 17 17 ...  0  0  0]
 ...
 [17 17 17 ...  0  0  0]
 [17 17 17 ...  0  0  0]
 [17 17 17 ...  0  0  0]], shape=(128, 132), dtype=int32)


In [37]:
  vocab_size = len(vocab)+1

  # The embedding dimension
  embedding_dim = 256

  # Number of RNN units
  rnn_units = 1024

  label_size = len(labels)  
  
  # build LSTM model
  def build_model(vocab_size,label_size, embedding_dim, rnn_units, batch_size):
        model = tf.keras.Sequential([
            tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None],mask_zero=True),
            tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
            tf.keras.layers.Dense(label_size)
            ])
        return model

  model = build_model(
        vocab_size = len(vocab)+1,
        label_size=len(labels)+1,
        embedding_dim=embedding_dim,
        rnn_units=rnn_units,
        batch_size=BATCH_SIZE)

  model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 256)          18944     
_________________________________________________________________
lstm (LSTM)                  (128, None, 1024)         5246976   
_________________________________________________________________
dense (Dense)                (128, None, 20)           20500     
Total params: 5,286,420
Trainable params: 5,286,420
Non-trainable params: 0
_________________________________________________________________


In [38]:
    import os

    # define loss function
    def loss(labels, logits):
        return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

    model.compile(optimizer='adam',
                  loss=loss,
                  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

    # Directory where the checkpoints will be saved
    checkpoint_dir = './training_checkpoints'
    # Name of the checkpoint files
    checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

    checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_prefix,
        save_weights_only=True)

In [39]:
    EPOCHS=20
    history = model.fit(ds_series_batch,
                        epochs=EPOCHS,
                        validation_data=ds_series_batch_valid,
                        callbacks=[checkpoint_callback])

Epoch 1/20
23/23 [==============================] - 388s 17s/step - loss: 0.2930 - sparse_categorical_accuracy: 0.8573 - val_loss: 0.2306 - val_sparse_categorical_accuracy: 0.8938
Epoch 2/20
23/23 [==============================] - 374s 16s/step - loss: 0.1951 - sparse_categorical_accuracy: 0.8862 - val_loss: 0.2282 - val_sparse_categorical_accuracy: 0.8938
Epoch 3/20
23/23 [==============================] - 371s 16s/step - loss: 0.1774 - sparse_categorical_accuracy: 0.8893 - val_loss: 0.2469 - val_sparse_categorical_accuracy: 0.8932
Epoch 4/20
23/23 [==============================] - 368s 16s/step - loss: 0.1712 - sparse_categorical_accuracy: 0.8930 - val_loss: 0.2008 - val_sparse_categorical_accuracy: 0.9000
Epoch 5/20
23/23 [==============================] - 382s 17s/step - loss: 0.1435 - sparse_categorical_accuracy: 0.8945 - val_loss: 0.1934 - val_sparse_categorical_accuracy: 0.9001
Epoch 6/20
23/23 [==============================] - 373s 16s/step - loss: 0.1338 - sparse_categorica

In [40]:
from sklearn.metrics import classification_report, confusion_matrix

preds = np.array([])
y_trues= np.array([])

# iterate through test set, make predictions based on trained model
for input_example_batch, target_example_batch in ds_series_batch_test:

  pred=model.predict_on_batch(input_example_batch)
  pred_max=tf.argmax(tf.nn.softmax(pred),2).numpy().flatten()
  y_true=target_example_batch.numpy().flatten()

  preds=np.concatenate([preds,pred_max])
  y_trues=np.concatenate([y_trues,y_true])

# remove padding from evaluation
remove_padding = [(p,y) for p,y in zip(preds,y_trues) if y!=0]

r_p = [x[0] for x in remove_padding]
r_t = [x[1] for x in remove_padding]

# print confusion matrix and classification report
print(confusion_matrix(r_p,r_t))
print(classification_report(r_p,r_t))

[[    0     0     0     0     9     0     0    24     0     0     1     0
      0     0     0     0     9     0     0]
 [    0     0   362     7     0     0     0     0     0     0     0     0
      0     0     7     0     4    12     0]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0]
 [    0     0     0    60     0     0     0     0     0     0     0     0
      0     0     0     0     3    13     0]
 [    0     0     0     0    76     0     0     8     0     4     0     0
      0     0     0     0     8     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0]
 [    0     0     0     1     0     0     0     0     0     0     0     0
      0     0     0     0     1     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0]
 [    0     0     0     0     0     0     0     

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
